In [28]:
# from google.colab import drive
# drive.mount('/content/drive/')

### Importation module

In [29]:
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import calendar
import matplotlib.pyplot as plt


### Connection with Colab / VSC

In [30]:
# colab
# quotes = pd.read_csv("/content/drive/MyDrive/Opply/Opply/d5d9348e5c87368cc1b74e6d4dde9794/Quotes.csv")
# request = pd.read_csv('/content/drive/MyDrive/Opply/Opply/d5d9348e5c87368cc1b74e6d4dde9794/Requests.csv')

# vsc

quotes = pd.read_csv("Quotes.csv")
request = pd.read_csv("Requests.csv")

# Exploration

In [31]:
print(request.shape)
print(quotes.shape)

(389, 4)
(492, 6)


In [32]:
request.head()

,Request UUID,Current Unit Price,Target Unit Price,Supply Priority
0,820412eb-8d4d-4d6c-a5e3-cfc977cd6d48,£3.50,£3.00,Price
1,08f1a913-1318-4656-9418-ce580a9871cb,£0.01,£0.01,Price
2,8cd6fd33-a1bc-43cb-9c63-df7d629d5362,£0.01,£0.01,Price
3,9388fbf6-410b-4f85-999a-1ff4d4cb733b,£0.01,£0.01,Price
4,37d05dfe-7f47-4eed-8e78-9f74c303bcad,£0.01,£0.01,Price


In [33]:
quotes.head()

,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided
0,94ba38d1-57ed-4a60-92d1-47b55a40a6dd,£5.10,Available,NaN,NaN,NaN
1,820412eb-8d4d-4d6c-a5e3-cfc977cd6d48,£3.50,Available,NaN,Yes,NaN
2,08f1a913-1318-4656-9418-ce580a9871cb,"£1,768.89",Available,NaN,Yes,Yes
3,4dedeeb1-12a5-4589-82ee-015e46d85790,£4.65,Declined,pricing: Too high,NaN,NaN
4,820412eb-8d4d-4d6c-a5e3-cfc977cd6d48,£13.00,Available,NaN,NaN,NaN


In [34]:
request.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Request UUID        389 non-null    object
 1   Current Unit Price  389 non-null    object
 2   Target Unit Price   389 non-null    object
 3   Supply Priority     287 non-null    object
dtypes: object(4)
memory usage: 12.3+ KB


In [35]:
quotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Request UUID        492 non-null    object
 1   Quote Unit Price    492 non-null    object
 2   Status              492 non-null    object
 3   Decline Reason      58 non-null     object
 4   Samples Available   275 non-null    object
 5   Documents Provided  279 non-null    object
dtypes: object(6)
memory usage: 23.2+ KB


* Identified unique values for each column.


In [ ]:
# Quotes
for col in quotes.columns:
  unique_value = quotes[col].unique()
  print(f"> : {col} ==> : {unique_value}")

  value_counts = quotes[col].value_counts(normalize= True).round(2) * 100
  value_counts_with_percent = value_counts.map("{:.2f} %".format)

  print(value_counts_with_percent)
  print(f'- {quotes[col].dtypes} -')
  print('---------------------------')

* 50 % of Decline Reason is : pricing 
  * to do : group the 'pricing:' and 'pricing' lines and analyze why ?

In [ ]:
# Requests
for col in request.columns:
  unique_value = request[col].unique()
  print(f"> : {col} ==> : {unique_value}")

  value_counts = request[col].value_counts(normalize= True).round(2) * 100
  value_counts_with_percent = value_counts.map("{:.2f} %".format)

  print(value_counts_with_percent)
  print(f'- {request[col].dtypes} -')
  print('---------------------------')

# Quotes :

## Cleaning :

In [38]:
quotes["Status"].value_counts()

Status
Expired             290
Available           124
Ordered              58
Declined             17
Contract Created      1
pending_payment       1
Proceeded             1
Name: count, dtype: int64

* replace nan by 'no' on column "Samples Available" and "Documents Provided"

In [39]:
quotes["Documents Provided"] = quotes["Documents Provided"].fillna('No')

In [40]:
quotes["Documents Provided"].isna().sum()

0

In [41]:
quotes["Samples Available"] = quotes["Samples Available"].fillna('No')

In [42]:
quotes["Samples Available"].isna().sum()

0

In [ ]:
# Quotes
for col in quotes.columns:
  unique_value = quotes[col].unique()
  print(f"> : {col} ==> : {unique_value}")

  value_counts = quotes[col].value_counts(normalize= True).round(2) * 100
  value_counts_with_percent = value_counts.map("{:.2f} %".format)

  print(value_counts_with_percent)
  print(f'- {quotes[col].dtypes} -')
  print('---------------------------')

## Analysis

* creation of new column **'decline reason new'** with 3 labels :
  * **not specified** : for all values nan
  * **pricing** : for all row of the reason mentionned pricing (I suppose that as soon as pricing is indicated, it means that the price is too high)
  * **no decline** : column 'Status' = 'Ordered'
  * **quote expired** : column 'Status' = 'Expired'
  * **others** : for other reason

In [85]:
# NaN by "not specified"
quotes["Decline Reason New"] = np.where(quotes['Decline Reason'].isna(), 'not specified', quotes["Decline Reason"])

# "pricing" 
quotes["Decline Reason New"] = np.where(quotes['Decline Reason New'].str.contains('pricing', case=False), 'pricing', quotes["Decline Reason New"])

# "no decline"
quotes["Decline Reason New"] = np.where(quotes['Status'] == 'Ordered', 'no decline', quotes["Decline Reason New"])

# "Expired"
quotes["Decline Reason New"] = np.where(quotes['Status'] == 'Expired', 'quote expired', quotes["Decline Reason New"])

# "others"
quotes["Decline Reason New"] = np.where(
    (
        (quotes["Decline Reason New"] != 'not specified') &
        (quotes["Decline Reason New"] != 'no decline') &
        (quotes["Decline Reason New"] != 'quote expired') &
        (~quotes["Decline Reason New"].str.contains('pricing', case=False))
    ),
    'others',
    quotes["Decline Reason New"]
)


In [86]:
# Repartion of Decline Reason New
quotes['Decline Reason New'].value_counts(normalize= True).round(2) * 100

Decline Reason New
quote expired    59.0
not specified    26.0
no decline       12.0
pricing           2.0
others            1.0
Name: proportion, dtype: float64

In [ ]:
quotes.head(50)

In [73]:
# Filter the dataframe based on the value of "status" == Ordered

quotes_ordered = quotes[quotes['Status'] == 'Ordered']
print(quotes_ordered.shape)
quotes_ordered.head()


(58, 7)


,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided,Decline Reason New
28,fca03c4f-3f16-40d0-b87c-16d9343e4a52,£4.65,Ordered,NaN,Yes,Yes,no decline
30,d596642b-c417-49e6-aa3b-ec6185fb2b9f,£0.58,Ordered,NaN,No,Yes,no decline
79,9cf11f62-7b1e-4a7f-9c40-c12ad721a72b,£1.66,Ordered,NaN,No,Yes,no decline
80,3c583881-a99d-42ca-a81b-648bc6884976,£1.58,Ordered,NaN,No,Yes,no decline
84,2473d2dc-0a5c-4a61-a73d-3050faf0f7a0,£3.77,Ordered,NaN,Yes,No,no decline
86,a12b7050-9288-4efc-aa22-ca0ca8e2d94a,£4.65,Ordered,NaN,No,Yes,no decline
88,26137052-6469-47ef-9ac2-a0fd353acdaf,£1.60,Ordered,NaN,Yes,Yes,no decline
90,141aeffb-21ce-418f-a9ab-76127b793f31,£7.26,Ordered,NaN,No,Yes,no decline
93,335e4424-c1f5-49ac-8e84-ac580076a926,£10.00,Ordered,NaN,No,No,no decline
136,fc24a58c-9cac-43a0-a198-136e8a89e2ba,£1.10,Ordered,NaN,Yes,Yes,no decline


* 11.79% of quotes are Ordered

In [44]:
# column : Samples Available = Yes / No in (%)

(quotes_ordered['Samples Available']).value_counts(normalize= True) * 100


Samples Available
No     60.344828
Yes    39.655172
Name: proportion, dtype: float64

In [45]:
# column : Documents Provided = Yes / No in (%)

(quotes_ordered['Documents Provided']).value_counts(normalize= True) * 100

Documents Provided
Yes    63.793103
No     36.206897
Name: proportion, dtype: float64

In [46]:
# column : Samples Available and Documents Provided = "Yes" in the same time

len(quotes_ordered[(quotes_ordered['Samples Available'] == 'Yes') & (quotes_ordered['Documents Provided'] == 'Yes')])


19

* On accepted quotes, approximately:
 * 40% have Samples Available
 * 36% have Documents Provided
 * 33% have both Samples Available and Documents Provided

### Merge quotes_order with Request



* understanding why a quote is accepted based on the
 * Target Unit Price
 * Current Unit Price
 * Quote Unit Price

---



In [47]:
# inner merge

print(quotes_ordered.shape)
quotes_ordered_request = quotes_ordered.merge(right= request, on= 'Request UUID', how = 'inner')
print(quotes_order_request.shape)
quotes_ordered_request.head()

(58, 6)


NameError: name 'quotes_order_request' is not defined

In [ ]:
# delete £ on columns 'price'

quotes_ordered_request['Quote Unit Price'] = quotes_ordered_request['Quote Unit Price'].replace('£', '', regex=True)
quotes_ordered_request['Current Unit Price'] = quotes_ordered_request['Current Unit Price'].replace('£', '', regex=True)
quotes_ordered_request['Target Unit Price'] = quotes_ordered_request['Target Unit Price'].replace('£', '', regex=True)

In [ ]:
quotes_ordered_request.head()

,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided,Current Unit Price,Target Unit Price,Supply Priority
0,fca03c4f-3f16-40d0-b87c-16d9343e4a52,4.65,Ordered,NaN,Yes,Yes,4.50,4.50,Price
1,d596642b-c417-49e6-aa3b-ec6185fb2b9f,0.58,Ordered,NaN,No,Yes,1.00,1.00,Price
2,9cf11f62-7b1e-4a7f-9c40-c12ad721a72b,1.66,Ordered,NaN,No,Yes,1.66,1.66,Price
3,3c583881-a99d-42ca-a81b-648bc6884976,1.58,Ordered,NaN,No,Yes,1.66,1.66,Price
4,2473d2dc-0a5c-4a61-a73d-3050faf0f7a0,3.77,Ordered,NaN,Yes,No,3.00,3.00,Price


In [ ]:
# convert columns 'price' to integer

columns_to_convert = ['Quote Unit Price', 'Current Unit Price', 'Target Unit Price']
quotes_ordered_request[columns_to_convert] = quotes_ordered_request[columns_to_convert].astype(float).round(2)


In [ ]:
quotes_ordered_request.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Request UUID        58 non-null     object 
 1   Quote Unit Price    58 non-null     float64
 2   Status              58 non-null     object 
 3   Decline Reason      0 non-null      object 
 4   Samples Available   58 non-null     object 
 5   Documents Provided  58 non-null     object 
 6   Current Unit Price  58 non-null     float64
 7   Target Unit Price   58 non-null     float64
 8   Supply Priority     39 non-null     object 
dtypes: float64(3), object(6)
memory usage: 4.5+ KB


In [ ]:
# delete rows with values = 0,01

quotes_ordered_request = quotes_ordered_request.loc[(quotes_ordered_request['Current Unit Price'] != 0.01) & (quotes_ordered_request['Target Unit Price'] != 0.01)]


* Addition of a column **'Price Difference (%)'** that calculates the percentage between the value requested by the buyer **Target Unit Price** and the price provided on the quote by the buyer **Quote Unit Price.**

In [ ]:
quotes_ordered_request['Price Difference (%)'] = ((quotes_ordered_request['Quote Unit Price'] - quotes_ordered_request['Target Unit Price']) / quotes_ordered_request['Target Unit Price']) * 100


<ipython-input-149-6c4dddc26466>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quotes_ordered_request['Price Difference (%)'] = ((quotes_ordered_request['Quote Unit Price'] - quotes_ordered_request['Target Unit Price']) / quotes_ordered_request['Target Unit Price']) * 100


In [ ]:
quotes_ordered_request.head()

,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided,Current Unit Price,Target Unit Price,Supply Priority,Price Difference (%)
0,fca03c4f-3f16-40d0-b87c-16d9343e4a52,4.65,Ordered,NaN,Yes,Yes,4.50,4.50,Price,3.333333
1,d596642b-c417-49e6-aa3b-ec6185fb2b9f,0.58,Ordered,NaN,No,Yes,1.00,1.00,Price,-42.000000
2,9cf11f62-7b1e-4a7f-9c40-c12ad721a72b,1.66,Ordered,NaN,No,Yes,1.66,1.66,Price,0.000000
3,3c583881-a99d-42ca-a81b-648bc6884976,1.58,Ordered,NaN,No,Yes,1.66,1.66,Price,-4.819277
4,2473d2dc-0a5c-4a61-a73d-3050faf0f7a0,3.77,Ordered,NaN,Yes,No,3.00,3.00,Price,25.666667


* We could add 2 more columns for:
 * Price negotiated upwards.
 * Price negotiated downwards.

### Request

In [ ]:
request.head()

,Request UUID,Current Unit Price,Target Unit Price,Supply Priority
0,820412eb-8d4d-4d6c-a5e3-cfc977cd6d48,£3.50,£3.00,Price
1,08f1a913-1318-4656-9418-ce580a9871cb,£0.01,£0.01,Price
2,8cd6fd33-a1bc-43cb-9c63-df7d629d5362,£0.01,£0.01,Price
3,9388fbf6-410b-4f85-999a-1ff4d4cb733b,£0.01,£0.01,Price
4,37d05dfe-7f47-4eed-8e78-9f74c303bcad,£0.01,£0.01,Price


In [ ]:
# delete £
request['Current Unit Price'] = request['Current Unit Price'].replace('£', '', regex=True)
request['Target Unit Price'] = request['Target Unit Price'].replace('£', '', regex=True)

In [ ]:
# delete rows with values = 0,01

request = request.loc[(request['Current Unit Price'] != '0.01') & (request['Target Unit Price'] != '0.01')]


In [ ]:
request.shape

(354, 4)

In [ ]:
# convert columns 'price' to integer

request[columns_to_convert] = request[columns_to_convert].replace({',': ''}, regex=True).astype(float).round(2)

columns_to_convert = ['Current Unit Price', 'Target Unit Price']
request[columns_to_convert] = request[columns_to_convert].astype(float).round(2)


In [ ]:
request.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354 entries, 0 to 388
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Request UUID        354 non-null    object 
 1   Current Unit Price  354 non-null    float64
 2   Target Unit Price   354 non-null    float64
 3   Supply Priority     258 non-null    object 
dtypes: float64(2), object(2)
memory usage: 13.8+ KB




---



* 29 dec : stop coding at 8:00. (2h)


---

